In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is 'last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
from random import sample
from collections import Counter, defaultdict

from tqdm import tqdm
from azure.cosmos.cosmos_client import CosmosClient

import sys
sys.path.append('../../../')  # CameraTraps repository base dir

from data_management.megadb.megadb_utils import MegadbUtils

# Useful queries

Example queries for the MegaDB's `sequences` table. MegaDB is a NoSQL database on Azure Cosmos DB.

Cosmos DB Python SDK (pre-release) documentation: https://azuresdkdocs.blob.core.windows.net/$web/python/azure-cosmos/4.0.0b5/index.html 

Subquery for nested objects: https://docs.microsoft.com/en-us/azure/cosmos-db/sql-query-subquery

## Connect to the Cosmos DB instance

`COSMOS_ENDPOINT` and `COSMOS_KEY` need to be environment variables. 

In [25]:
# Initialize Cosmos DB client
url = os.environ['COSMOS_ENDPOINT']
key = os.environ['COSMOS_KEY']
client = CosmosClient(url, credential=key)

database = client.get_database_client('camera-trap')
container_sequences = database.get_container_client('sequences')

## Get the `datasets` table

which records the location and access levels of each dataset.

In [26]:
%%time
megadb_utils = MegadbUtils()
datasets = megadb_utils.get_datasets_table()

CPU times: user 38.5 ms, sys: 5.2 ms, total: 43.7 ms
Wall time: 1.08 s


#### List the public datasets

In [27]:
public_datasets = [d['dataset_name'] for d in datasets.values() if 'public' in d['access']]
public_datasets

['caltech', 'wcs', 'nacti', 'snapshotserengeti', 'bellevue_190602']

## Examples

In the examples, we limit the selection to a few entries using the `TOP` keyword. When using the DB to create datasets, delete the TOP keyword and arg.

### How many entries are in a dataset?

In [52]:
%%time

query = '''
SELECT COUNT(seq) as count
FROM sequences seq
'''

partition_key = 'sulross_kitfox'  # use None if querying across all partitions

if partition_key is not None:
    result_iterable = container_sequences.query_items(query=query,
                                                      partition_key=partition_key)
else:
    result_iterable = container_sequences.query_items(query=query,
                                                      enable_cross_partition_query=True)
    
for res in result_iterable:
    print(res)

{'count': 140900}
CPU times: user 18.9 ms, sys: 2.83 ms, total: 21.7 ms
Wall time: 403 ms


### View some entries from a dataset

In [53]:
%%time

query = '''
SELECT TOP 10 *
FROM sequences seq
'''

partition_key = 'sulross_kitfox'  # use None if querying across all partitions

if partition_key is not None:
    result_iterable = container_sequences.query_items(query=query,
                                                      partition_key=partition_key)
else:
    result_iterable = container_sequences.query_items(query=query,
                                                      enable_cross_partition_query=True)

results = list(result_iterable)

print('Length of results:', len(results))

Length of results: 10
CPU times: user 7.06 ms, sys: 2.34 ms, total: 9.4 ms
Wall time: 143 ms


### What datasets are there that actually have data?

In [8]:
%%time

result_iterable = container_sequences.query_items(
    query='''
SELECT DISTINCT seq.dataset
FROM sequences seq
''',
    enable_cross_partition_query=True
)

results = [item['dataset'] for item in result_iterable]

print('Length of results:', len(results))

Length of results: 23
CPU times: user 452 ms, sys: 45.1 ms, total: 497 ms
Wall time: 5min 8s


### Image entries in a dataset with class "empty" - demonstrating JOIN
Can use `partition_key` for this query without needing joins

Refer to https://docs.microsoft.com/en-us/azure/cosmos-db/sql-query-join - query is constructed as follows (thanks to Chris Ritchie):
1. Iterate through each document (seq) (query scoped to a single partition otherwise you could add an additional WHERE clause for seq.dataset = 'zsl_borneo'): `FROM sequences seq`
2. For each document (seq) expand each child element in the images array (im): `im IN seq.images`
3. Apply a cross product with the root of the item (seq) with each child element (im) the second step flattened
4. Project each child element (im): `JOIN`

In [22]:
%%time

query = '''
SELECT TOP 200 im.file, seq.dataset, seq.location
FROM sequences seq JOIN im IN seq.images 
WHERE ARRAY_LENGTH(im.class) = 1 AND ARRAY_CONTAINS(im.class, "empty") 
OR ARRAY_LENGTH(seq.class) = 1 AND ARRAY_CONTAINS(seq.class, "empty") 
'''

# WHERE ARRAY_LENGTH(im.class) > 0 AND ARRAY_CONTAINS(im.class, "empty")

result_iterable = container_sequences.query_items(query, enable_cross_partition_query=True)

results = list(result_iterable)

print('Length of results:', len(results))

Length of results: 20000
CPU times: user 1.32 s, sys: 119 ms, total: 1.44 s
Wall time: 29.5 s


### All bounding box annotations

Including those sent to annotation and confirmed empty

In [27]:
%%time

# location should always be on the sequence level
query = '''
SELECT TOP 10 im.bbox, im.file, seq.dataset, seq.location
FROM sequences seq JOIN im IN seq.images 
WHERE ARRAY_LENGTH(im.bbox) >= 0
'''

partition_key = 'caltech'  # use None if querying across all partitions

if partition_key is not None:
    result_iterable = container_sequences.query_items(query=query,
                                                      partition_key=partition_key)
else:
    result_iterable = container_sequences.query_items(query=query,
                                                      enable_cross_partition_query=True)

results = list(result_iterable)

print('Length of results:', len(results))

Length of results: 10
CPU times: user 5.56 ms, sys: 1.7 ms, total: 7.26 ms
Wall time: 62.5 ms


### All sequences where at least one image has bbox annotation

These are sequences with at least one image that was sent for bbox annotation, which could have come back confirmed empty.

Do not use join here; will result in duplicated sequence objects.

In [ ]:
%%time

# location should always be on the sequence level
query = '''
SELECT TOP 10 seq
FROM sequences seq
WHERE (SELECT VALUE COUNT(im) FROM im IN seq.images WHERE ARRAY_LENGTH(im.bbox) >= 0) > 0
'''

partition_key = 'caltech'  # use None if querying across all partitions

if partition_key is not None:
    result_iterable = container_sequences.query_items(query=query,
                                                      partition_key=partition_key)
else:
    result_iterable = container_sequences.query_items(query=query,
                                                      enable_cross_partition_query=True)

results = list(result_iterable)

print('Length of results:', len(results))

### All images with the specified species at the image level

In [26]:
%%time

species_requested = 'horse-tailed squirrel'

query ='''
SELECT TOP 10 im.class, im.file
FROM im IN sequences.images 
WHERE ARRAY_LENGTH(im.class) > 0 AND ARRAY_CONTAINS(im.class, "{}")
'''.format(species_requested)

result_iterable = container_sequences.query_items(query, enable_cross_partition_query=True)

results = list(result_iterable)

print('Length of results:', len(results))

Length of results: 10
CPU times: user 10.8 ms, sys: 2.63 ms, total: 13.4 ms
Wall time: 177 ms


### Species count where the label is at the sequence level

In [12]:
%%time

query = '''
SELECT seq.class
FROM sequences seq
WHERE ARRAY_LENGTH(seq.class) > 0
'''

result_iterable = container_sequences.query_items(query, enable_cross_partition_query=True)

species_seq = Counter()
for item in result_iterable:
    res = item['class']
    species_seq.update(res)

CPU times: user 7min 48s, sys: 42.3 s, total: 8min 30s
Wall time: 2h 51min 52s


In [14]:
len(species_seq)

430

### Species count where the label is at the image level

In [29]:
%%time

query = '''
SELECT TOP 100000 im.class
FROM im IN sequences.images 
WHERE ARRAY_LENGTH(im.class) > 0
'''

result_iterable = container_sequences.query_items(query, enable_cross_partition_query=True)

species_im = Counter()
for item in result_iterable:
    res = item['class']
    species_im.update(res)

CPU times: user 5.17 s, sys: 573 ms, total: 5.75 s
Wall time: 1min 55s


We can increase the throughput of the database when doing large queries to get response faster.

In [ ]:
species_im

### Total number of sequence entries in database

In [31]:
%%time

query = '''
SELECT VALUE COUNT(1)
FROM seq
'''

result_iterable = container_sequences.query_items(query, enable_cross_partition_query=True)

for seq_count in result_iterable:
    print(seq_count)

146526
CPU times: user 13 ms, sys: 2.45 ms, total: 15.4 ms
Wall time: 175 ms


### Total number of images in all sequences

In [32]:
%%time

query = '''
SELECT VALUE COUNT(1)
FROM im IN sequences.images 
'''

result_iterable = container_sequences.query_items(query, enable_cross_partition_query=True)

for im_count in result_iterable:
    print(im_count)

596773
CPU times: user 37.8 ms, sys: 5.76 ms, total: 43.6 ms
Wall time: 30.3 s


### List last inserted sequences based on insertion timestap

In [33]:
%%time

query = '''
SELECT TOP 10 seq.dataset, seq._ts, seq.seq_id
FROM sequences seq
ORDER BY seq._ts DESC
'''

result_iterable = container_sequences.query_items(query, enable_cross_partition_query=True)

results = list(result_iterable)

CPU times: user 12 ms, sys: 3.14 ms, total: 15.1 ms
Wall time: 176 ms


In [34]:
results

[{'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50869'},
 {'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50870'},
 {'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50871'},
 {'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50872'},
 {'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50873'},
 {'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50874'},
 {'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50875'},
 {'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50876'},
 {'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50877'},
 {'dataset': 'zsl_borneo', '_ts': 1573253030, 'seq_id': '50878'}]

### Locations in each dataset

Not yet useful.

In [75]:
%%time

query = '''
SELECT TOP 20000 seq.location, seq.dataset
FROM sequences seq
WHERE seq.location != null
'''

result_iterable = container_sequences.query_items(query, enable_cross_partition_query=True)

results = defaultdict(set)

for item in result_iterable:
    #if 'location' in item:
    results[item['dataset']].add(item['location'])

CPU times: user 1.24 s, sys: 128 ms, total: 1.36 s
Wall time: 25.3 s
